In [4]:
# 참조문서 수정

import pymysql
import json

# DB 설정
USER = "root"
PASS = "1234"
HOST = "127.0.0.1"
PORT = 3306
DB = "any_approval"

# DB 연결
conn = pymysql.connect(
    host=HOST,
    port=PORT,
    user=USER,
    password=PASS,
    database=DB,
    charset='utf8mb4'
)

def safe_json_loads(text_data):
    """안전하게 JSON 파싱"""
    if text_data is None:
        return []
    if isinstance(text_data, str):
        text_data = text_data.strip()
        if not text_data or text_data == '[]':
            return []
    try:
        return json.loads(text_data)
    except:
        return []

def get_source_id_from_title(cursor, title):
    """제목으로 source_id 찾기"""
    if not title or title.strip() == '':
        return None
    
    query = "SELECT source_id FROM documents WHERE title = %s LIMIT 1"
    cursor.execute(query, (title,))
    result = cursor.fetchone()
    
    return result['source_id'] if result else None

def convert_references_to_json(cursor, references_text):
    """텍스트 references를 JSON 형식으로 변환"""
    if not references_text or references_text.strip() == '':
        return []
    
    # 이미 JSON 형식이면 그대로 파싱
    try:
        parsed = json.loads(references_text)
        if isinstance(parsed, list):
            return parsed
    except:
        pass
    
    # 텍스트면 source_id 찾아서 변환
    source_id = get_source_id_from_title(cursor, references_text)
    if source_id:
        return [{"sourceId": source_id}]
    
    return []

try:
    cursor = conn.cursor(pymysql.cursors.DictCursor)
    
    # end_year가 2020인 문서만 조회 (백틱으로 예약어 처리)
    query = """
    SELECT id, source_id, doc_num, `references`, doc_type, title, `attaches`, 
           drafter_name, drafter_position, drafter_dept, drafter_email, drafter_dept_code,
           created_at, created_date, doc_body, doc_status, `referrers`, activities, 
           form_name, is_public, end_year
    FROM documents 
    WHERE end_year = 2020
    """
    cursor.execute(query)
    rows = cursor.fetchall()
    
    # 첫 번째 행의 references 값 확인 (디버깅)
    if rows:
        print(f"🔍 첫 번째 행 references 원본 값: {repr(rows[0]['references'])}")
        print(f"🔍 첫 번째 행 references 타입: {type(rows[0]['references'])}")
    
    # cmds 파일 생성
    with open('documents_2020.cmds', 'w', encoding='utf-8') as f:
        for idx, row in enumerate(rows):
            # references 변환
            references_json = convert_references_to_json(cursor, row['references'])
            
            # JSON 구조 생성
            doc = {
                "sourceId": row['source_id'],
                "docNum": row['doc_num'].replace(' ', '') if row['doc_num'] else row['doc_num'],
                "references": references_json,
                # "htmlDate": row['created_date'].strftime('%Y-%m-%d %H:%M:%S') if row['created_date'] else None,
                "docType": row['doc_type'],
                "title": row['title'],
                "attaches": safe_json_loads(row['attaches']),
                "drafter": {
                    "positionName": row['drafter_position'],
                    "deptName": row['drafter_dept'],
                    "name": row['drafter_name'],
                    "emailId": row['drafter_email'],
                    "deptCode": row['drafter_dept_code']
                },
                "createdAt": row['created_at'],
                "docBody": row['doc_body'],
                "docStatus": row['doc_status'],
                "referrers": safe_json_loads(row['referrers']),
                "activities": safe_json_loads(row['activities']),
                "formName": row['form_name'],
                "isPublic": bool(row['is_public'])
            }
            
            # 첫 번째 문서만 확인 출력
            if idx == 0:
                print(f"🔍 변환된 references: {references_json}")
            
            # addDocument 형식으로 작성
            f.write(f"addDocument {json.dumps(doc, ensure_ascii=False, separators=(',', ':'))}\n")
    
    print(f"✅ 2020년 문서 총 {len(rows)}개를 documents_2020.cmds 파일로 저장했습니다.")
    
except Exception as e:
    print(f"❌ 에러 발생: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    cursor.close()
    conn.close()

🔍 첫 번째 행 references 원본 값: '[]'
🔍 첫 번째 행 references 타입: <class 'str'>
🔍 변환된 references: []
✅ 2020년 문서 총 7704개를 documents_2020.cmds 파일로 저장했습니다.
